In [1]:
import gradio as gr
from transformers import pipeline
import numpy as np
import openai
import warnings
warnings.filterwarnings('ignore')
import os
from dotenv import load_dotenv
load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")


C:\Users\c_mar\anaconda3\envs\dev\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
# Preference
# Initialize the ASR pipeline
transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")

def transcribe_and_save(audio):
    sr, y = audio
    y = y.astype(np.float32)
    y /= np.max(np.abs(y))
    transcription = transcriber({"sampling_rate": sr, "raw": y})["text"]
    
    # Write the transcription to a file
    with open("transcription_movie_selection.txt", "w") as file:
        file.write(transcription)
    
    return transcription

# Create the Gradio interface
demo = gr.Interface(
    transcribe_and_save,
    gr.Audio(sources=["microphone"]),
    "text",
)

demo.launch(share=False)
#change to True for outside URL


Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


In [5]:
#pure movie selection analysis with OpenAI
#preprocess txt file

with open('transcription_movie_selection.txt', 'r', encoding="utf-8") as file:
    text = file.read()


In [7]:
# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

# Set your OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

# Function time

def get_preference(question):
    messages = [
        {"role": "system", "content": "Determine three movies to make the person happier based on the provided text."},
        {"role": "user", "content": question}
    ]
    
    # Use the chat completions method
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=messages
    )
    
    # Extract the answer from the response
    answer = response.choices[0].message.content
    return answer

def analyze_document(filename):
    with open(filename, 'r') as file:
        text = file.read()
    preference = get_preference(text)
    print(f"Movie Recommendations: {preference}")

analyze_document('transcription_movie_selection.txt')


Movie Recommendations: 1. "Lilo & Stitch" - This animated movie features a lovable, mischievous alien and the heartfelt bond he forms with a young girl named Lilo. It's full of laughter and endearing moments that will surely lift your spirits.

2. "The Secret Life of Pets" - This is a delightful movie that showcases the unexpected adventures of domesticated dogs when their owners are away. It’s humorous and touching, instilling a sense of joy and fondness for our furry friends.

3. "Ice Cream Man" - An offbeat, 90s horror comedy which has now become a cult classic. For someone who can appreciate some dark humor, this quirky movie featuring an ice cream man can serve as an oddball cheer-up film. 

Alternatively, if you're looking for something more lighthearted with the ice cream theme, "Cloudy with a Chance of Meatballs 2" has a scene with adorable, living, and unforgettable foodimals which do include cheerful ice cream creatures.
